In [1]:
import os
import json
import h5py
import faiss
import heapq
import numpy as np
import pandas as pd
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict

## Parameters

In [2]:
input_file = "20240401-065-03_0228-0236_t00s00"
metric = "specdiff"
dataset = "20250110"
embeddings_file = f"../data/tables/{dataset}/{metric}.h5"
index_file = f"../data/tables/{dataset}/{metric}.faiss"
segmented_dir = f"../data/datasets/{dataset}/segmented"
augmented_dir = f"../data/datasets/{dataset}/augmented"
graph_dir = f"../data/datasets/{dataset}/graphs"
candidate_count = 3  # number of neighbors from different tracks to find

## Load Data

In [3]:
with h5py.File(embeddings_file, "r") as f:
    embeddings = [e / np.linalg.norm(e, keepdims=True) for e in f["embeddings"]]
    filenames = [str(name[0], "utf-8") for name in f["filenames"]]
df = pd.DataFrame({"embeddings": embeddings}, index=filenames, dtype=np.float32)
df.head()

,embeddings
20231220-080-01_0000-0005_t00s00,"(-0.018957129, -0.054159317, 0.025529064, 0.02..."
20231220-080-01_0000-0005_t00s01,"(-0.01733134, -0.03869735, 0.03737718, 0.01761..."
20231220-080-01_0000-0005_t00s02,"(-0.020138163, -0.047744356, 0.029153284, 0.01..."
20231220-080-01_0000-0005_t00s03,"(-0.01579446, -0.042559903, 0.03145476, 0.0272..."
20231220-080-01_0000-0005_t00s04,"(-0.021046124, -0.052375674, 0.025457665, 0.01..."


In [4]:
faiss_index = faiss.read_index(index_file)
faiss_index.ntotal

545088

In [5]:
grouped_files = defaultdict(list)

for filename in os.listdir(augmented_dir):
    if filename.endswith(".mid"):
        track_name = filename.split("_")[0]
        grouped_files[track_name].append(filename.split(".")[0])  # + "_t00s00")

for track, files in grouped_files.items():
    print(f"Track: {track}")
    for file in files:
        print(f"  {file}")
    break

Track: 20240511-050-01
  20240511-050-01_0086-0095_t00s07
  20240511-050-01_0326-0335_t11s02
  20240511-050-01_0335-0345_t01s04
  20240511-050-01_0614-0623_t02s01
  20240511-050-01_0028-0038_t04s02
  20240511-050-01_0211-0220_t07s02
  20240511-050-01_0527-0537_t07s00
  20240511-050-01_0556-0566_t07s02
  20240511-050-01_0566-0575_t08s05
  20240511-050-01_0374-0383_t02s05
  20240511-050-01_0009-0019_t05s02
  20240511-050-01_0000-0009_t10s06
  20240511-050-01_0710-0719_t11s02
  20240511-050-01_0691-0700_t02s04
  20240511-050-01_0153-0163_t07s06
  20240511-050-01_0383-0393_t03s00
  20240511-050-01_0153-0163_t09s03
  20240511-050-01_0643-0652_t02s03
  20240511-050-01_0105-0115_t05s03
  20240511-050-01_0556-0566_t09s07
  20240511-050-01_0527-0537_t09s05
  20240511-050-01_0566-0575_t06s00
  20240511-050-01_0211-0220_t09s07
  20240511-050-01_0239-0249_t01s05
  20240511-050-01_0719-0729_t01s03
  20240511-050-01_0451-0460_t10s00
  20240511-050-01_0316-0326_t10s00
  20240511-050-01_0124-0134_t04s

## helper functions

In [6]:
def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    """Computes cosine similarity between two vectors."""
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))


def cosine_distance(file1: str, file2: str) -> float:
    """Computes cosine distance (1 - cosine similarity)."""
    return 1 - cosine_similarity(
        np.asarray(df.loc[file1, "embeddings"], dtype=np.float32),
        np.asarray(df.loc[file2, "embeddings"], dtype=np.float32),
    )

In [7]:
def draw_graph(graph_path: str):
    G = nx.node_link_graph(json.load(open(graph_path)))
    pos = nx.spring_layout(G, seed=42)
    plt.figure(figsize=(8, 6))
    options = {
        "node_color": "blue",
        "node_size": 20,
        "edge_color": "grey",
        "linewidths": 0,
        "width": 0.1,
    }
    nx.draw_networkx(G, pos, with_labels=False, **options)
    nx.draw_networkx_nodes(G, pos)
    # nx.draw_networkx_edges(G, pos)
    # nx.draw_networkx_labels(G, pos)
    # nx.draw_networkx_edge_labels(
    #     G,
    #     pos,
    #     edge_labels={(u, v): f"{d['weight']:.2f}" for u, v, d in G.edges(data=True)},
    # )
    plt.title(f"{track}")
    plt.axis("off")
    plt.show()


for graph_json in os.listdir(graph_dir)[:3]:
    graph_path = os.path.join(graph_dir, graph_json)
    draw_graph(graph_path)

In [8]:
# generate graph for each track
# Create a new graph
track_graphs = {}

for track, files in grouped_files.items():
    G = nx.Graph(name=track)

    # add nodes
    for file in tqdm(files, "adding nodes"):
        G.add_node(file)

    # add edges
    for i in tqdm(range(len(files)), "adding edges"):
        for j in range(i + 1, len(files)):
            G.add_edge(files[i], files[j], weight=cosine_distance(files[i], files[j]))

    track_graphs[track] = G

    with open(f"{graph_dir}/{track}.json", "w") as f:
        json.dump(nx.node_link_data(G), f)

adding edges:   3%|▎         | 227/7296 [01:02<32:35,  3.62it/s]


KeyboardInterrupt: 

## build path

### find nearest neighbors from different tracks

In [ ]:
# get elements for search
input_track = input_file.split("_")[0]
input_embedding = np.asarray(
    df.loc[input_file, "embeddings"], dtype=np.float32
).reshape(1, -1)

# get neighbors
distances, indices = faiss_index.search(input_embedding, 5000)
neighbor_files = df.index[indices.flatten()].tolist()

# get nearest neighbors with different track
unique_tracks = set()
nearest_neighbors = []
for file in neighbor_files:
    track = file.split("_")[0]
    if track != input_track and track not in unique_tracks:
        unique_tracks.add(track)
        nearest_neighbors.append(file)
    if len(nearest_neighbors) >= 10:
        break

print(nearest_neighbors)

In [ ]:
graph = nx.Graph()

input_embedding = np.asarray(
    df.loc[input_file, "embeddings"], dtype=np.float32
).reshape(1, -1)

neighbor_embeddings = np.stack(df.loc[nearest_neighbors, "embeddings"].values)
similarities, indices = faiss_index.search(input_embedding, len(nearest_neighbors))

print("Adding edges to the graph based on similarity scores...")
for i, neighbor in enumerate(nearest_neighbors):
    similarity = similarities[0][i]
    cost = 1 - similarity  # transform similarity into cost
    graph.add_edge(input_file, neighbor, weight=cost)

In [ ]:
# Create a mapping from track names to colors
tracks = sorted({node.split("_")[0] for node in graph.nodes()})
color_map = plt.cm.get_cmap("tab10", len(tracks))
track_to_color = {track: color_map(i) for i, track in enumerate(tracks)}

# Assign a color to each node based on its track
node_colors = [track_to_color[node.split("_")[0]] for node in graph.nodes()]

# Compute layout and plot the graph
pos = nx.spring_layout(graph, seed=42)
plt.figure(figsize=(8, 6))
nx.draw_networkx(graph, pos, node_color=node_colors, with_labels=True, font_size=8)
nx.draw_networkx_nodes(graph, pos)
nx.draw_networkx_edges(graph, pos)
nx.draw_networkx_labels(graph, pos)
nx.draw_networkx_edge_labels(
    graph, pos, edge_labels={(u, v): d["weight"] for u, v, d in graph.edges(data=True)}
)
plt.axis("off")
plt.show()

In [ ]:
def build_faiss_index(embeddings: np.ndarray, num_neighbors: int = 20):
    """Builds a FAISS index for fast nearest-neighbor lookup."""
    index = faiss.IndexFlatIP(
        embeddings.shape[1]
    )  # Inner product for cosine similarity
    faiss.normalize_L2(embeddings)  # Normalize for cosine similarity
    index.add(embeddings)
    return index


def astar_knn_search(
    embeddings: np.ndarray,
    index: faiss.IndexFlatIP,
    start_idx: int,
    goal_idx: int,
    k: int = 10,
):
    """
    Finds an approximate shortest path between start_idx and goal_idx using A* search with k-NN graph.

    Args:
        embeddings (np.ndarray): The dataset of embeddings.
        index (faiss.IndexFlatIP): Pre-built FAISS index for fast lookup.
        start_idx (int): Index of the starting node.
        goal_idx (int): Index of the goal node.
        k (int): Number of nearest neighbors to explore at each step.

    Returns:
        list: The sequence of node indices forming the path from start to goal.
    """
    start_embedding = embeddings[start_idx]
    goal_embedding = embeddings[goal_idx]

    # Priority queue (min-heap) for A*
    open_set = []
    heapq.heappush(
        open_set, (cosine_distance(start_embedding, goal_embedding), 0, start_idx, [])
    )

    # Track visited nodes and costs
    g_scores = {start_idx: 0}
    visited = set()

    while open_set:
        _, cost, current_idx, path = heapq.heappop(open_set)

        if current_idx in visited:
            continue

        # Add to visited set
        visited.add(current_idx)
        path = path + [current_idx]

        # Goal reached
        if current_idx == goal_idx:
            return path

        # Get nearest neighbors dynamically
        distances, indices = index.search(embeddings[current_idx][None, :], k)
        neighbors = zip(indices[0], distances[0])

        for neighbor_idx, _ in neighbors:
            if neighbor_idx in visited:
                continue

            tentative_g_score = cost + cosine_distance(
                embeddings[current_idx], embeddings[neighbor_idx]
            )
            if (
                neighbor_idx not in g_scores
                or tentative_g_score < g_scores[neighbor_idx]
            ):
                g_scores[neighbor_idx] = tentative_g_score
                f_score = tentative_g_score + cosine_distance(
                    embeddings[neighbor_idx], goal_embedding
                )
                heapq.heappush(
                    open_set, (f_score, tentative_g_score, neighbor_idx, path)
                )

    return None  # No path found


# path = astar_knn_search(embeddings, index, start_idx, goal_idx, k=20)
# print(f"Path from {start_idx} to {goal_idx}: {path}")

In [ ]:
# # Create a new graph
# similarity_graph = nx.Graph()

# # Get all embeddings
# all_embeddings = np.stack(df["embeddings"].values)

# # Compute similarities between all pairs of embeddings
# distances, indices = faiss_index.search(all_embeddings, len(all_embeddings))

# # Add edges to the graph based on cosine similarity
# for i, neighbors in enumerate(indices):
#     for j, neighbor in enumerate(neighbors):
#         if i != neighbor:  # Avoid self-loops
#             similarity = 1 - distances[i][j]  # Convert distance to similarity
#             similarity_graph.add_edge(
#                 df.index[i], df.index[neighbor], weight=similarity
#             )

# # Create a mapping from track names to colors
# tracks = sorted({node.split("_")[0] for node in similarity_graph.nodes()})
# color_map = plt.cm.get_cmap("tab10", len(tracks))
# track_to_color = {track: color_map(i) for i, track in enumerate(tracks)}

# # Assign a color to each node based on its track
# node_colors = [track_to_color[node.split("_")[0]] for node in similarity_graph.nodes()]

# # Compute layout and plot the graph
# pos = nx.spring_layout(similarity_graph, seed=42)
# plt.figure(figsize=(12, 10))
# nx.draw_networkx(
#     similarity_graph, pos, node_color=node_colors, with_labels=True, font_size=8
# )
# nx.draw_networkx_nodes(similarity_graph, pos)
# nx.draw_networkx_edges(similarity_graph, pos)
# nx.draw_networkx_labels(similarity_graph, pos)
# nx.draw_networkx_edge_labels(
#     similarity_graph,
#     pos,
#     edge_labels={
#         (u, v): f"{d['weight']:.2f}" for u, v, d in similarity_graph.edges(data=True)
#     },
# )
# plt.axis("off")
# plt.show()